In [45]:
import pandas as pd
import numpy as np

In [46]:
df = pd.read_csv("~/Downloads/toy_data.csv")

In [3]:
# df.dropna(axis = 1, thresh= 0.9*df.shape[0], inplace= True)
# nunique = df.apply(pd.Series.nunique)
# cols_to_drop = nunique[nunique == 1].index
# df.drop(cols_to_drop, axis = 1, inplace= True)

In [4]:
#required_cols = ['agency_name','agency_abbr','agency_code','state_name','state_abbr','state_code','loan_amount_000s', 'applicant_sex_name', 'applicant_sex', 'tract_to_msamd_income', 'population','census_tract_number','number_of_1_to_4_family_units','minority_population', 'applicant_income_000s']

In [57]:
required_cols = ['applicant_sex', 'applicant_sex_name']

In [58]:
df = df[required_cols]

In [59]:
nunique = df.apply(pd.Series.nunique)
discrete_columns = nunique[nunique < 100].index

In [60]:
numeric_columns = df.select_dtypes(include=['number']).columns
object_columns = df.select_dtypes(include=['object']).columns
df[numeric_columns] = df[numeric_columns].fillna(0)
df[object_columns] = df[object_columns].fillna("NA")

df.isna().any()

applicant_sex         False
applicant_sex_name    False
dtype: bool

In [91]:
from importlib import reload
import synthesizer,transformer,sampler,conditional,models
reload(models)
reload(sampler)
reload(synthesizer)
reload(transformer)
reload(conditional)
data = df.loc[0:100000,:]
model = synthesizer.CTGANSynthesizer(embedding_dim = 128, batch_size=500)
model.fit(data, discrete_columns, 20,log_frequency= False)


Epoch 1, Loss G: -0.2574, Loss D: 0.0696
Epoch 2, Loss G: -0.3005, Loss D: 0.0477
Epoch 3, Loss G: -0.3971, Loss D: -0.2095
Epoch 4, Loss G: -0.3722, Loss D: 0.0559
Epoch 5, Loss G: -0.1695, Loss D: -0.0251
Epoch 6, Loss G: -0.3195, Loss D: -0.1138
Epoch 7, Loss G: -0.1799, Loss D: -0.0456
Epoch 8, Loss G: -0.1611, Loss D: -0.0316
Epoch 9, Loss G: -0.3961, Loss D: -0.2208
Epoch 10, Loss G: -0.3810, Loss D: -0.0401
Epoch 11, Loss G: -0.4531, Loss D: 0.0211
Epoch 12, Loss G: -0.3502, Loss D: -0.0168
Epoch 13, Loss G: -0.4086, Loss D: -0.1081
Epoch 14, Loss G: -0.3447, Loss D: -0.1095
Epoch 15, Loss G: -0.1126, Loss D: 0.0568
Epoch 16, Loss G: -0.2125, Loss D: 0.3906
Epoch 17, Loss G: 0.2036, Loss D: -0.0262
Epoch 18, Loss G: -0.0169, Loss D: -0.0110
Epoch 19, Loss G: -0.0345, Loss D: -0.1554
Epoch 20, Loss G: 0.0625, Loss D: 0.0095


In [94]:
sampled = model.sample(100000)

In [88]:
agency_abbr = np.array(["CFPB", "HUD", "FDIC", "FRS", "NCUA", "OCC"])
applicant_sex = np.array([2, 3, 1,4])

In [95]:
og = (pd.crosstab(data.applicant_sex_name, data.applicant_sex).reindex(applicant_sex, axis="columns")/data.shape[0] * 100).rename(index = {"Information not provided by applicant in mail, Internet, or telephone application" : "Not provided"})
og['Total'] = og.sum(axis=1)
og.loc['Total']= og.sum()
og = og.round(2)
og

applicant_sex,2,3,1,4,Total
applicant_sex_name,,,,,
Female,27.06,0.00,0.00,0.00,27.06
Not provided,0.00,14.62,0.00,0.00,14.62
Male,0.00,0.00,58.28,0.00,58.28
Not applicable,0.00,0.00,0.00,0.05,0.05
Total,27.06,14.62,58.28,0.05,100.00


In [96]:
new = (pd.crosstab(sampled.applicant_sex_name, sampled.applicant_sex).reindex(applicant_sex, axis="columns")/sampled.shape[0] * 100).rename(index = {"Information not provided by applicant in mail, Internet, or telephone application" : "Not provided"})
new['Total'] = new.sum(axis=1)
new.loc['Total']= new.sum()
new = new.round(2)
new

applicant_sex,2,3,1,4,Total
applicant_sex_name,,,,,
Female,9.92,0.57,3.54,1.04,15.07
Not provided,0.23,1.98,0.91,0.26,3.38
Male,3.23,1.50,70.93,1.70,77.35
Not applicable,0.86,0.79,1.88,0.67,4.20
Total,14.24,4.84,77.26,3.66,100.00


In [215]:
new - og

applicant_sex,2,3,1,4,Total
applicant_sex_name,,,,,
Female,-1.29,0.51,0.74,0.04,0.0
Not provided,0.40,-0.62,0.22,0.01,0.0
Male,0.90,0.24,-1.20,0.06,0.0
Not applicable,0.01,0.00,0.01,-0.03,0.0
Total,0.02,0.13,-0.23,0.07,0.0


In [216]:
og = (pd.crosstab(data.applicant_sex_name,data.agency_abbr).reindex(agency_abbr, axis="columns")/data.shape[0] * 100).rename(index = {"Information not provided by applicant in mail, Internet, or telephone application" : "Not provided"})
og['Total'] = og.sum(axis=1)
og.loc['Total']= og.sum()
og = og.round(2)
og

agency_abbr,CFPB,HUD,FDIC,FRS,NCUA,OCC,Total
applicant_sex_name,,,,,,,
Female,6.00,14.84,2.09,0.79,2.36,0.97,27.06
Not provided,3.30,8.36,0.97,0.32,1.26,0.41,14.62
Male,13.38,31.63,4.96,1.82,4.08,2.39,58.28
Not applicable,0.01,0.01,0.01,0.00,0.01,0.00,0.05
Total,22.70,54.84,8.04,2.94,7.70,3.78,100.00


In [217]:
new = (pd.crosstab(sampled.applicant_sex_name,sampled.agency_abbr).reindex(agency_abbr, axis="columns")/sampled.shape[0] * 100).rename(index = {"Information not provided by applicant in mail, Internet, or telephone application" : "Not provided"})
new['Total'] = new.sum(axis=1)
new.loc['Total']= new.sum()
new = new.round(2)
new

agency_abbr,CFPB,HUD,FDIC,FRS,NCUA,OCC,Total
applicant_sex_name,,,,,,,
Female,6.09,8.22,0.91,4.17,6.68,0.99,27.06
Not provided,5.45,5.62,0.44,0.73,2.24,0.14,14.62
Male,12.94,29.79,10.26,0.63,1.20,3.46,58.28
Not applicable,0.01,0.01,0.01,0.00,0.01,0.01,0.05
Total,24.49,43.64,11.62,5.53,10.12,4.60,100.00


In [218]:
new - og

agency_abbr,CFPB,HUD,FDIC,FRS,NCUA,OCC,Total
applicant_sex_name,,,,,,,
Female,0.09,-6.62,-1.18,3.38,4.32,0.02,0.0
Not provided,2.15,-2.74,-0.53,0.41,0.98,-0.27,0.0
Male,-0.44,-1.84,5.30,-1.19,-2.88,1.07,0.0
Not applicable,0.00,0.00,0.00,0.00,0.00,0.01,0.0
Total,1.79,-11.20,3.58,2.59,2.42,0.82,0.0


In [164]:
og = (pd.crosstab(data.agency_abbr,data.applicant_sex).reindex(applicant_sex, axis="columns")/data.shape[0] * 100)
og['Total'] = og.sum(axis=1)
og.loc['Total']= og.sum()
og = og.round(2)
og

applicant_sex,2,3,1,4,Total
agency_abbr,,,,,
CFPB,6.00,3.30,13.38,0.01,22.70
FDIC,2.09,0.97,4.96,0.01,8.04
FRS,0.79,0.32,1.82,0.00,2.94
HUD,14.84,8.36,31.63,0.01,54.84
NCUA,2.36,1.26,4.08,0.01,7.70
OCC,0.97,0.41,2.39,0.00,3.78
Total,27.06,14.62,58.28,0.05,100.00


In [155]:
new = (pd.crosstab(sampled.agency_abbr,sampled.applicant_sex).reindex(applicant_sex, axis="columns")/sampled.shape[0] * 100)
new['Total'] = new.sum(axis=1)
new.loc['Total']= new.sum()
new = new.round(2)
new

applicant_sex,2,3,1,4,Total
agency_abbr,,,,,
CFPB,6.25,5.68,12.43,0.04,24.40
FDIC,0.92,0.47,10.00,0.06,11.45
FRS,4.29,0.78,0.56,0.01,5.65
HUD,8.68,5.74,29.12,0.06,43.59
NCUA,6.86,2.24,1.24,0.03,10.38
OCC,0.95,0.16,3.42,0.01,4.54
Total,27.95,15.07,56.77,0.20,100.00


In [104]:
new - og

applicant_sex,2,3,1,4,Total
agency_abbr,,,,,
CFPB,-2.65,-2.17,-0.82,0.09,-5.57
FDIC,0.68,-0.55,-0.22,0.03,-0.07
FRS,0.09,0.02,1.47,0.03,1.60
HUD,-0.76,-0.70,0.38,0.16,-0.92
NCUA,-0.86,-0.46,3.52,0.04,2.25
OCC,0.41,0.41,1.83,0.06,2.69
Total,-3.10,-3.44,6.14,0.40,0.00


In [80]:
from importlib import reload
import synthesizer,transformer,sampler,conditional,models
reload(models)
reload(sampler)
reload(synthesizer)
reload(transformer)
reload(conditional)
data = df.loc[0:100000,:]
model1 = synthesizer.CTGANSynthesizer(embedding_dim = 128, batch_size=500)
model1.fit(data, discrete_columns, 30,log_frequency= False)
model2 = synthesizer.CTGANSynthesizer(embedding_dim = 128, batch_size=500)
model2.fit(data, discrete_columns, 30,log_frequency= True)


 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]


KeyboardInterrupt: 

In [156]:
sampled = pd.concat([model1.sample(50000),model2.sample(50000)], axis = 0)

In [176]:
original_applicant_sex_p = (data.applicant_sex_name.value_counts()/data.shape[0]).to_numpy()
applicant_sex_values = (data.applicant_sex_name.value_counts()/data.shape[0]).index.to_numpy()
original_agency_abbr_p = (data.agency_abbr.value_counts()/data.shape[0]).to_numpy()
agency_abbr_values = (data.agency_abbr.value_counts()/data.shape[0]).index.to_numpy()


In [211]:
sampled_data_list = []
sampled_all = model2.sample(500000)
for name,value in zip(applicant_sex_values, original_applicant_sex_p):
    temp = sampled_all[sampled_all['applicant_sex_name'] == name]
    sampled_data_list.append(temp.sample(n= int(value*100000)))



In [212]:
sampled = pd.concat(sampled_data_list,axis = 0 )

In [203]:
sampled_data_list = []
sampled_all = model2.sample(500000)
for name,value in zip(agency_abbr_values, original_agency_abbr_p):
    temp = sampled_all[sampled_all['agency_abbr'] == name]
    sampled_data_list.append(temp.sample(n= int(value*100000)))


In [204]:
sampled = pd.concat(sampled_data_list,axis = 0 )

In [17]:
del1 = np.array([[-3.6]])

In [18]:
x0 = np.array([[1],[-0.9]])

In [19]:
x0*np.transpose(del1)

array([[-3.6 ],
       [ 3.24]])

In [8]:
x0

array([[-0.44,  0.88],
       [-0.88,  1.75]])

In [20]:
k = [1,2,3]

In [28]:
k[-2:]

[2, 3]

In [76]:
p = np.array([[0.1, 0.2], [0.43, 0.4]])

In [63]:
i = np.random.choice(2, 300)

array([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1])

In [78]:
p


array([[0.1 , 0.2 ],
       [0.43, 0.4 ]])